In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
#just the usual
import os
import numpy as np
import pandas as pd

#All necessary plotly libraries
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import yfinance as yf
import time


In [4]:
df.keys()

NameError: name 'df' is not defined

In [6]:
while True:
    df = yf.download(tickers='^NSEI', period='1d', interval='1m',
                    auto_adjust=True, prepost=True)
    fig = go.Figure([go.Scatter(x=df.index, y=df['Close'])])
    # Add titles
    fig.update_layout(
        title='NSEI Closing Price',
        yaxis_title='NIFTY50')
    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=5, label="5m", step="minute", stepmode="backward"),
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=1, label="HTD", step="hour", stepmode="todate"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()
    py.offline.plot(fig, filename='./templates/prev5.html',auto_open=False)


    #EMA
    df['EMA_9'] = df['Close'].ewm(9).mean().shift()
    df['EMA_22'] = df['Close'].ewm(22).mean().shift()
    #SMA
    df['SMA_5'] = df['Close'].rolling(5).mean().shift()
    df['SMA_10'] = df['Close'].rolling(10).mean().shift()
    df['SMA_15'] = df['Close'].rolling(15).mean().shift()
    df['SMA_30'] = df['Close'].rolling(30).mean().shift()
    #RSI14


    def RSI(df, n=14):
        close = df['Close']
        delta = close.diff()
        delta = delta[1:]
        pricesUp = delta.copy()
        pricesDown = delta.copy()
        pricesUp[pricesUp < 0] = 0
        pricesDown[pricesDown > 0] = 0
        rollUp = pricesUp.rolling(n).mean()
        rollDown = pricesDown.abs().rolling(n).mean()
        rs = rollUp / rollDown
        rsi = 100.0 - (100.0 / (1.0 + rs))
        return rsi

        df['RSI'] = RSI(df).fillna(0)


    EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
    EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
    df['MACD'] = pd.Series(EMA_12 - EMA_26)
    df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df.EMA_9, name='EMA 9'))
    fig.add_trace(go.Scatter(x=df.index, y=df.EMA_22, name='EMA 22'))
    fig.add_trace(go.Scatter(x=df.index, y=df.SMA_5, name='SMA 5'))
    fig.add_trace(go.Scatter(x=df.index, y=df.SMA_10, name='SMA 10'))
    fig.add_trace(go.Scatter(x=df.index, y=df.SMA_15, name='SMA 15'))
    fig.add_trace(go.Scatter(x=df.index, y=df.SMA_30, name='SMA 30'))
    fig.add_trace(go.Scatter(x=df.index, y=df.Close, name='Close', opacity=0.3))
    # Add titles
    fig.update_layout(
        title='NSEI live price EMA and SMA indicators',
        yaxis_title='NIFTY50')
    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=5, label="5m", step="minute", stepmode="backward"),
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=1, label="HTD", step="hour", stepmode="todate"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()
    py.offline.plot(fig, filename='./templates/EMA.html', auto_open=False)


    fig = make_subplots(rows=2, cols=1)
    fig.add_trace(go.Scatter(x=df.index, y=df.Close, name='Close'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=EMA_12, name='EMA 12'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=EMA_26, name='EMA 26'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(
        x=df.index, y=df['MACD_signal'], name='Signal line'), row=2, col=1)
    # Add titles
    fig.update_layout(
        title='NSEI live price EMA + MACD + Signal lines',
        yaxis_title='NIFTY50')

    fig.show()
    py.offline.plot(fig, filename='./templates/EMAMACD.html', auto_open=False)
    time.sleep(10)



[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

'./templates/EMA.html'

'./templates/EMAMACD.html'